# Task 2 CNN
### Question 1
First we import all the necessary libraries 
- os - for getting the image path 
- numpy - for handling data 
-cv2 - for resizing image 
-scikit-learn - for train test split and measuring accuracy 
-tensorflow - for CNN 
-datetime - for saving the model 


In [ ]:
import os
import numpy as np
import cv2
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Activation, Flatten, Conv1D, MaxPooling1D
from sklearn.metrics import accuracy_score
import datetime
from tensorflow.keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img

Now, we create ImageDataGenerator object for image augmentation. We only have around 40 images of each sample which is very less for training a neural network. So, I used Image Augmentation(which generates more images by scaling, rotating, etc. the original images) for generating more samples so that the model doesn't have the problem of overfitting. 

The images are increased from 40 per category to around 280 per category, making the total images around 17000.

In [ ]:
datagen = ImageDataGenerator(
        rotation_range=40,
        width_shift_range=0.2,
        height_shift_range=0.2,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=False, 
        vertical_flip=False,
        fill_mode='nearest')

This code block uses the os library to get the list of all the folders in the train folder(Sample 001, Sample 002...) and stores them in a list names files.

This will be used while augmentation and while importing the images.

In [ ]:
files = []
for f in os.listdir('train/'):
    if os.path.isdir(f'train/{f}'):
        files.append(f'train/{f}')
files.sort()

This block of code creates the augmented images. It create 6 images from each sample, thus images become 280(40*6 + 40(original)).

In [ ]:
for i in range(1):
    img_path = os.listdir(files[i])
    img_path.sort()
    label = i
    for j in img_path:
        path = f'{files[i]}/{j}'
        img = load_img(path)  # this is a PIL image
        x = img_to_array(img)  # this is a Numpy array with shape (3, 150, 150)
        x = x.reshape((1,) + x.shape)  # this is a Numpy array with shape (1, 3, 150, 150)
    a = 0
    for batch in datagen.flow(x, batch_size=1, save_to_dir=files[i], save_prefix=f'{a}', save_format='png'):
        a += 1
        if a > 5:
            break

Now, we import all the images and their labels and store them in X and y respectively. To import images, I have used OpenCV and imported them in grayscale.

In [ ]:
X = []
y = []
for i in range(len(files)):
    img_path = os.listdir(files[i])
    img_path.sort()
    label = i
    for j in img_path:
        path = f'{files[i]}/{j}'
        img = cv2.imread(path, cv2.IMREAD_GRAYSCALE)
        X.append(img)
        y.append(label)
size = len(y)

Now, we preprocess the data. I have reduced the image size to 50 x 50 for faster training.

In [ ]:
height = 50
width = 50
for i in range(size):
    img = X[i]
    img_resize = cv2.resize(img, (height, width))
    X[i] = img_resize

This block does a few things.

First line, changes X from list to a numpy array. Next line, scales the data, it changes the data range from 0-255 to 0-1, this sometimes increases the accuracy and it's always a good practice to scale data.

Next, we split the data into test and training set. We use 10% of the data for testing(validation and testing).

Last two lines are used to convert y_train and y_test samples to categorical samples by one hot encoding.

In [ ]:
X = np.stack(X)  
X = X/255.0
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.1)
y_train = to_categorical(y_train, dtype = 'int64') # One hot encoding
y_test1 = to_categorical(y_test, dtype = 'int64') # One hot encoding

Now we define the model. By trial and error and all the different approaches mentioned in the experiment log, I have finalised this model. It gives 96% training accuracy and 89% testing accuracy.

I am using 3 convolution layers, 3 maxpooling layers, 2 dropout layers for preventing overfitting, two dense layers for getting the features and for output, and one flatten layer. 

relu activation is used for all layers except output layer which uses softmax. \
Loss function used is Categorical Crossentropy.\
Adam optimizer is used after testing with RMSProp, Adam and SGD.

In [ ]:
model = Sequential()
model.add(Conv1D(32,kernel_size = (5), padding ='valid',activation = 'relu', input_shape = (height, width)))
model.add(MaxPooling1D(pool_size = (2)))

model.add(Conv1D(64,kernel_size = (5), padding ='same',activation = 'relu'))
model.add(MaxPooling1D(pool_size = (2)))

model.add(Conv1D(64,kernel_size = (3), padding ='same',activation = 'relu'))
model.add(MaxPooling1D(pool_size = (2)))

model.add(Dropout(0.05))

model.add(Flatten())

model.add(Dense(128, activation='relu'))

model.add(Dropout(0.1))

model.add(Dense(62, activation = 'softmax'))

model.compile(loss = 'categorical_crossentropy', optimizer = 'adam', metrics = ['accuracy'])
print("Model Defined. Training now.")

Now we train the model with 60 epochs. I have seen that by 50 epochs the model reaches convergence as its accuracy stops increasing and pretty much occilates around 95%.

In [ ]:
model.fit(X_train, y_train, epochs = 60, validation_data=(X_test, y_test1))
print("Model trained!")

Now, we predict the test samples and store them in y_pred, then we convert them back from categorical to number, and we use accuracy_score metric to judge our model.

In [ ]:
y_pred = model.predict(X_test)
idx = np.argmax(y_pred, axis = 1)

X_test = np.stack(X_test)  
y_test = np.stack(y_test)
results = accuracy_score(y_test, idx)
print("Accuracy = ", results)

This block of code saves the model in the format(model_datetime of creation_accuracy)

In [ ]:
time = datetime.datetime.now()
model.save(f"model_{time}_{int(100*results)}.h5")